# How to call functions with chat models

This notebook covers how to use the Chat Completions API in combination with external functions to extend the capabilities of GPT models.

## How to use functions

`functions` is an optional parameter in the ChatCompletion API which can be used to provide function specifications. The purpose of this is to enable models to generate outputs which adhere to function input schemas. Note that the API will not actually execute any function calls. It is up to developers to execute function calls using model outputs.

If the `functions` parameter is provided then by default the model will decide when it is appropriate to use one of the functions. The API can also be forced to use a specific function by setting the `function_call` parameter to `{"name": "<insert-function-name>"}`. If a function is used, the output will contain `"finish_reason": "function_call"` in the response, as well as a `function_call` object that has the name of the function and the generated function arguments.

Functions are specified with the following fields:

- **Name:** The name of the function.
- **Description:** A description of what the function does. The model will use this to decide when to call the function.
- **Parameters:** The parameters object contains all of the input fields the function requires. These inputs can be of the following types: String, Number, Boolean, Object, Null, AnyOf. Refer to the [API reference docs](https://platform.openai.com/docs/api-reference/chat) for details.
- **Required:** Which of the parameters are required to make a query. The rest will be treated as optional.

You can chain function calls by executing the function and passing the output of the function execution directly back to the assistant. This can lead to _infinite loop_ behaviour where the model continues calling functions indefinitely, however guardrails can be put in place to prevent this.

## Walkthrough

This cookbook takes you through the following workflow:

- **Basic concepts:** Creating an example function and getting the API to use it if appropriate.
- **Integrating API calls with function execution:** Creating an agent that uses API calls to generate function arguments and then executes the function.
- **Using multiple functions:** Allowing multiple functions to be called in sequence before responding to the user.


In [1]:
import arxiv
import ast
import concurrent
from csv import writer
from IPython.display import display, Markdown, Latex
import json
import openai
import os
import pandas as pd
from PyPDF2 import PdfReader
import requests
from scipy import spatial
from tenacity import retry, wait_random_exponential, stop_after_attempt
import tiktoken
from tqdm import tqdm
from termcolor import colored

%load_ext autoreload
%autoreload 2

GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"

## Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [2]:
from agentai.api import chat_complete
from agentai.conversation import Conversation
from agentai.function_parser import function_info

## Basic concepts

Next we'll create a specification for a function called ```get_current_weather```. Later we'll pass this function specification to the API in order to generate function arguments that adhere to the specification.

In [3]:
from enum import Enum

class TemperatureUnit(Enum):
    celsius = "celsius"
    fahrenheit = "fahrenheit"

@function_info
def get_current_weather(location: str, format: TemperatureUnit) -> str:
    """
    Get the current weather

    Args:
        location (str): The city and state, e.g. San Francisco, CA
        format (str): The temperature unit to use. Infer this from the users location.

    Returns:
        str: The current weather
    """
    # Your function implementation goes here.
    return ""



registered_functions = [get_current_weather]
functions = [json.loads(func.json_info) for func in registered_functions]

In [18]:
conversation = Conversation()
conversation.add_message("user", "what is the weather like today?")

message = {'role': 'assistant', 'content': None}
chat_response = chat_complete(conversation.conversation_history, functions=functions, model=GPT_MODEL)
message = chat_response.json()["choices"][0]["message"]
while message["content"] is None:
    print("No response from assistant, trying again")
    chat_response = chat_complete(conversation.conversation_history, functions=functions, model=GPT_MODEL)
    message = chat_response.json()["choices"][0]["message"]
conversation.add_message(message["role"], message["content"])
message

No response from assistant, trying again
No response from assistant, trying again
No response from assistant, trying again
No response from assistant, trying again
No response from assistant, trying again
No response from assistant, trying again


{'role': 'assistant',
 'content': 'Which city and state would you like to know the weather for?'}

In [20]:
conversation.conversation_history

[{'role': 'user', 'content': 'what is the weather like today?'},
 {'role': 'assistant',
  'content': 'Which city and state would you like to know the weather for?'}]

In [21]:
# Once the user provides the required information, the model can generate the function arguments
conversation.add_message("user", "I'm in Bengaluru, India")
chat_response = chat_complete(conversation.conversation_history, functions=functions, model=GPT_MODEL)

chat_response.json()["choices"][0]
# chat_response.json()

{'index': 0,
 'message': {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'get_current_weather',
   'arguments': '{\n  "location": "Bengaluru, India",\n  "format": "celsius"\n}'}},
 'finish_reason': 'function_call'}

## Integrating API calls with function execution

In our next example, we'll demonstrate how to execute functions whose inputs are model-generated, and use this to implement an agent that can answer questions for us about a database. For simplicity we'll use the [Chinook sample database](https://www.sqlitetutorial.net/sqlite-sample-database/).

*Note:* SQL generation use cases are high-risk in a production environment - models can be unreliable when generating consistent SQL syntax. A more reliable way to solve this problem may be to build a query generation API that takes the desired columns as input from the model.

### Pull SQL Database Info

First let's define some helpful utility functions to extract data from a SQLite database.

In [22]:
import sqlite3

conn = sqlite3.connect("../data/Chinook.db")
print("Opened database successfully")

Opened database successfully


Now can use these utility functions to extract a representation of the database schema.

In [23]:
from agentai.sqlite_utils import DBUtils
database_schema_dict = DBUtils(conn).get_database_info()
database_schema_string = "\n".join(
    [f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}" for table in database_schema_dict]
)

In [24]:
print(database_schema_string)

Table: Album
Columns: AlbumId, Title, ArtistId
Table: Artist
Columns: ArtistId, Name
Table: Customer
Columns: CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId
Table: Employee
Columns: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email
Table: Genre
Columns: GenreId, Name
Table: Invoice
Columns: InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total
Table: InvoiceLine
Columns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity
Table: MediaType
Columns: MediaTypeId, Name
Table: Playlist
Columns: PlaylistId, Name
Table: PlaylistTrack
Columns: PlaylistId, TrackId
Table: Track
Columns: TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice


As before, we'll define a function specification for the function we'd like the API to generate arguments for. Notice that we are inserting the database schema into the function specification. This will be important for the model to know about.

In [25]:
@function_info
def ask_database(query:str) -> str:
    """
    Use this function to answer user questions about music. Input should be a fully formed SQL query.

    Args:
        query (str):SQL query extracting info to answer the user's question.
                    SQL should be written using this database schema:
                    Table: Album
                    Columns: AlbumId, Title, ArtistId
                    Table: Artist
                    Columns: ArtistId, Name
                    Table: Customer
                    Columns: CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId
                    Table: Employee
                    Columns: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email
                    Table: Genre
                    Columns: GenreId, Name
                    Table: Invoice
                    Columns: InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total
                    Table: InvoiceLine
                    Columns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity
                    Table: MediaType
                    Columns: MediaTypeId, Name
                    Table: Playlist
                    Columns: PlaylistId, Name
                    Table: PlaylistTrack
                    Columns: PlaylistId, TrackId
                    Table: Track
                    Columns: TrackId, Name, AlbumId, MediaTypeId, GenreId, Composer, Milliseconds, Bytes, UnitPrice
                    
                    IMPORTANT: Please return a fixed SQL in PLAIN TEXT.
                    Your response should consist of ONLY the SQL query.

    Returns:
        str: _description_
    """
    try:
        results = conn.execute(query).fetchall()
        return results
    except Exception as e:
        raise Exception(f"SQL error: {e}")

agentai_functions = [json.loads(func.json_info) for func in [ask_database]]

### SQL execution

Now let's implement the function that the agent will use to query the database. We also need to implement utilities to integrate the calls to the Chat Completions API with the function it is calling.

In [59]:
from agentai.api import chat_complete_execute_fn

In [61]:
agent_system_message = """You are ChinookGPT, a helpful assistant who gets answers to user questions from the Chinook Music Database.
Provide as many details as possible to your users
Begin!"""

sql_conversation = Conversation()
sql_conversation.add_message(role="system", content=agent_system_message)
sql_conversation.add_message("user", "Hi, who are the top 5 artists by number of tracks")
assistant_message = chat_complete_execute_fn(sql_conversation, agentai_functions, GPT_MODEL, ask_database)

In [62]:
sql_conversation.display_conversation(detailed=True)

system: You are ChinookGPT, a helpful assistant who gets answers to user questions from the Chinook Music Database.
Provide as many details as possible to your users
Begin!


user: Hi, who are the top 5 artists by number of tracks


function: [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]


assistant: The top 5 artists by the number of tracks are:

1. Iron Maiden - 213 tracks
2. U2 - 135 tracks
3. Led Zeppelin - 114 tracks
4. Metallica - 112 tracks
5. Lost - 92 tracks




In [63]:
sql_conversation.add_message("user", "What is the name of the album with the most tracks")
chat_response = chat_complete_execute_fn(sql_conversation, agentai_functions, GPT_MODEL, ask_database)

In [64]:
sql_conversation.display_conversation(detailed=True)

system: You are ChinookGPT, a helpful assistant who gets answers to user questions from the Chinook Music Database.
Provide as many details as possible to your users
Begin!


user: Hi, who are the top 5 artists by number of tracks


function: [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]


assistant: The top 5 artists by the number of tracks are:

1. Iron Maiden - 213 tracks
2. U2 - 135 tracks
3. Led Zeppelin - 114 tracks
4. Metallica - 112 tracks
5. Lost - 92 tracks


user: What is the name of the album with the most tracks


function: [('Greatest Hits', 57)]


assistant: The album with the most tracks is called "Greatest Hits" and it has 57 tracks.




## Using Multiple Functions

Now let's construct a scenario in which we provide a model with more than one function to call. We'll create an agent that uses data from arXiv to answer questions about academic subjects. It has two new functions at its disposal:
- **get_articles**: A function that gets arXiv articles on a subject and summarizes them for the user with links.
- **read_article_and_summarize**: This function takes one of the previously searched articles, reads it in its entirety and summarizes the core argument, evidence and conclusions.

This will get you comfortable with a multi-function workflow that can choose from multiple services, and where some of the data from the first function is persisted to be used by the second.

### arXiv search

We'll first set up some utilities that will underpin our two functions.

Downloaded papers will be stored in a directory (we use ```./data/papers``` here). We create a file ```arxiv_library.csv``` to store the embeddings and details for downloaded papers to retrieve against using ```summarize_text```.

In [ ]:
# Set a directory to store downloaded papers
data_dir = os.path.join(os.curdir, "data", "papers")
paper_dir_filepath = "./data/arxiv_library.csv"

# Generate a blank dataframe where we can store downloaded files
df = pd.DataFrame(list())
df.to_csv(paper_dir_filepath)

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def embedding_request(text):
    response = openai.Embedding.create(input=text, model=EMBEDDING_MODEL)
    return response


def get_articles(query, library=paper_dir_filepath, top_k=5):
    """This function gets the top_k articles based on a user's query, sorted by relevance.
    It also downloads the files and stores them in arxiv_library.csv to be retrieved by the read_article_and_summarize.
    """
    search = arxiv.Search(query=query, max_results=top_k, sort_by=arxiv.SortCriterion.Relevance)
    result_list = []
    for result in search.results():
        result_dict = {}
        result_dict.update({"title": result.title})
        result_dict.update({"summary": result.summary})

        # Taking the first url provided
        result_dict.update({"article_url": [x.href for x in result.links][0]})
        result_dict.update({"pdf_url": [x.href for x in result.links][1]})
        result_list.append(result_dict)

        # Store references in library file
        response = embedding_request(text=result.title)
        file_reference = [
            result.title,
            result.download_pdf(data_dir),
            response["data"][0]["embedding"],
        ]

        # Write to file
        with open(library, "a") as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(file_reference)
            f_object.close()
    return result_list

In [ ]:
# Test that the search is working
result_output = get_articles("ppo reinforcement learning")
result_output[0]

In [ ]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100,
) -> list[str]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = embedding_request(query)
    query_embedding = query_embedding_response["data"][0]["embedding"]
    strings_and_relatednesses = [
        (row["filepath"], relatedness_fn(query_embedding, row["embedding"])) for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n]

In [ ]:
def read_pdf(filepath):
    """Takes a filepath to a PDF and returns a string of the PDF's contents"""
    # creating a pdf reader object
    reader = PdfReader(filepath)
    pdf_text = ""
    page_number = 0
    for page in reader.pages:
        page_number += 1
        pdf_text += page.extract_text() + f"\nPage Number: {page_number}"
    return pdf_text


# Split a text into smaller chunks of size n, preferably ending at the end of a sentence
def create_chunks(text, n, tokenizer):
    """Returns successive n-sized chunks from provided text."""
    tokens = tokenizer.encode(text)
    i = 0
    while i < len(tokens):
        # Find the nearest end of sentence within a range of 0.5 * n and 1.5 * n tokens
        j = min(i + int(1.5 * n), len(tokens))
        while j > i + int(0.5 * n):
            # Decode the tokens and check for full stop or newline
            chunk = tokenizer.decode(tokens[i:j])
            if chunk.endswith(".") or chunk.endswith("\n"):
                break
            j -= 1
        # If no end of sentence found, use n tokens as the chunk size
        if j == i + int(0.5 * n):
            j = min(i + n, len(tokens))
        yield tokens[i:j]
        i = j


def extract_chunk(content, template_prompt):
    """This function applies a prompt to some input content. In this case it returns a summarize chunk of text"""
    prompt = template_prompt + content
    response = openai.ChatCompletion.create(
        model=GPT_MODEL, messages=[{"role": "user", "content": prompt}], temperature=0
    )
    return response["choices"][0]["message"]["content"]


def summarize_text(query):
    """This function does the following:
    - Reads in the arxiv_library.csv file in including the embeddings
    - Finds the closest file to the user's query
    - Scrapes the text out of the file and chunks it
    - Summarizes each chunk in parallel
    - Does one final summary and returns this to the user"""

    # A prompt to dictate how the recursive summarizations should approach the input paper
    summary_prompt = (
        """Summarize this text from an academic paper. Extract any key points with reasoning.\n\nContent:"""
    )

    # If the library is empty (no searches have been performed yet), we perform one and download the results
    library_df = pd.read_csv(paper_dir_filepath).reset_index()
    if len(library_df) == 0:
        print("No papers searched yet, downloading first.")
        get_articles(query)
        print("Papers downloaded, continuing")
        library_df = pd.read_csv(paper_dir_filepath).reset_index()
    library_df.columns = ["title", "filepath", "embedding"]
    library_df["embedding"] = library_df["embedding"].apply(ast.literal_eval)
    strings = strings_ranked_by_relatedness(query, library_df, top_n=1)
    print("Chunking text from paper")
    pdf_text = read_pdf(strings[0])

    # Initialise tokenizer
    tokenizer = tiktoken.get_encoding("cl100k_base")
    results = ""

    # Chunk up the document into 1500 token chunks
    chunks = create_chunks(pdf_text, 1500, tokenizer)
    text_chunks = [tokenizer.decode(chunk) for chunk in chunks]
    print("Summarizing each chunk of text")

    # Parallel process the summaries
    with concurrent.futures.ThreadPoolExecutor(max_workers=len(text_chunks)) as executor:
        futures = [executor.submit(extract_chunk, chunk, summary_prompt) for chunk in text_chunks]
        with tqdm(total=len(text_chunks)) as pbar:
            for _ in concurrent.futures.as_completed(futures):
                pbar.update(1)
        for future in futures:
            data = future.result()
            results += data

    # Final summary
    print("Summarizing into overall summary")
    response = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[
            {
                "role": "user",
                "content": f"""Write a summary collated from this collection of key points extracted from an academic paper.
                        The summary should highlight the core argument, conclusions and evidence, and answer the user's query.
                        User query: {query}
                        The summary should be structured in bulleted lists following the headings Core Argument, Evidence, and Conclusions.
                        Key points:\n{results}\nSummary:\n""",
            }
        ],
        temperature=0,
    )
    return response

In [ ]:
# Test the summarize_text function works
chat_test_response = summarize_text("PPO reinforcement learning sequence generation")

In [ ]:
print(chat_test_response["choices"][0]["message"]["content"])

### Configure Agent

We'll now create 2 function specifications for functions that provide access to the arXiv data. We'll also create some more utilities to integrate Chat Completions API calls with function execution.

In [ ]:
# Initiate our get_articles and read_article_and_summarize functions
arxiv_functions = [
    {
        "name": "get_articles",
        "description": """Use this function to get academic papers from arXiv to answer user questions.""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            User query in JSON. Responses should be summarized and should include the article URL reference
                            """,
                }
            },
            "required": ["query"],
        },
        "name": "read_article_and_summarize",
        "description": """Use this function to read whole papers and provide a summary for users.
        You should NEVER call this function before get_articles has been called in the conversation.""",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            Description of the article in plain text based on the user's query
                            """,
                }
            },
            "required": ["query"],
        },
    }
]

In [ ]:
def chat_completion_with_function_execution(messages, functions=[None]):
    """This function makes a ChatCompletion API call with the option of adding functions"""
    response = chat_completion_request(messages, functions)
    full_message = response.json()["choices"][0]
    if full_message["finish_reason"] == "function_call":
        print(f"Function generation requested, calling function")
        return call_arxiv_function(messages, full_message)
    else:
        print(f"Function not required, responding to user")
        return response.json()


def call_arxiv_function(messages, full_message):
    """Function calling function which executes function calls when the model believes it is necessary.
    Currently extended by adding clauses to this if statement."""

    if full_message["message"]["function_call"]["name"] == "get_articles":
        try:
            parsed_output = json.loads(full_message["message"]["function_call"]["arguments"])
            print("Getting search results")
            results = get_articles(parsed_output["query"])
        except Exception as e:
            print(parsed_output)
            print(f"Function execution failed")
            print(f"Error message: {e}")
        messages.append(
            {
                "role": "function",
                "name": full_message["message"]["function_call"]["name"],
                "content": str(results),
            }
        )
        try:
            print("Got search results, summarizing content")
            response = chat_completion_request(messages)
            return response.json()
        except Exception as e:
            print(type(e))
            raise Exception("Function chat request failed")

    elif full_message["message"]["function_call"]["name"] == "read_article_and_summarize":
        parsed_output = json.loads(full_message["message"]["function_call"]["arguments"])
        print("Finding and reading paper")
        summary = summarize_text(parsed_output["query"])
        return summary

    else:
        raise Exception("Function does not exist and cannot be called")

### arXiv conversation

Let's test out our function in conversation

In [ ]:
# Start with a system message
paper_system_message = """You are arXivGPT, a helpful assistant pulls academic papers to answer user questions.
You summarize the papers clearly so the customer can decide which to read to answer their question.
You always provide the article_url and title so the user can understand the name of the paper and click through to access it.
Begin!"""
paper_conversation = Conversation()
paper_conversation.add_message("system", paper_system_message)

In [ ]:
# Add a user message
paper_conversation.add_message("user", "Hi, how does PPO reinforcement learning work?")
chat_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
paper_conversation.add_message("assistant", assistant_message)
display(Markdown(assistant_message))

In [ ]:
# Add another user message to induce our system to use the second tool
paper_conversation.add_message(
    "user",
    "Can you read the PPO sequence generation paper for me and give me a summary",
)
updated_response = chat_completion_with_function_execution(
    paper_conversation.conversation_history, functions=arxiv_functions
)
display(Markdown(updated_response["choices"][0]["message"]["content"]))